In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# リスト3.3.15 同義語の定義（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・日本語検索向けに「検索時のみ」同義語展開を行うアナライザ "jpn-search" を定義し、
#     インデックス jp_index を作成する。
#   ・同義語は「すし」「スシ」「鮨」「寿司」を相互同値として扱い、表記ゆれを吸収する。
#
# 理論メモ:
#   ・synonym フィルタは既定で **等価展開（expand=true 相当）** を行う。
#       "A, B, C" 形式は A↔B↔C を相互に展開（どれで検索しても他が候補に加わる）。
#     片方向の正規化（例: 略語→正規形）を行いたい場合は "A, B => C" 形式や synonyms_graph を検討する。
#   ・**検索側のみで同義語**を適用し、**索引側は素直に保持**するのが一般的な設計。
#     - 理由: 観測性（ログで元の語が見える）、運用性（辞書更新時に再インデックスを避けやすい）。
#   ・同義語の更新はしばしば **インデックス再作成＋reindex→エイリアス切替** が最も安全。
#     ホットリロードに制約があるため、変更計画は運用手順に組み込むこと。
#   ・トークナイズ順序: char_filter → tokenizer（kuromoji）→ filter（synonyms→原形化→POS/stop→数値→長音）。
#     先に正規化・形態素解析を済ませた語彙に対して同義語展開が掛かる想定。
#   ・注意: "user_dictionary" は **Elasticsearch ノード側のパス**。ローカルファイルではない。
# -----------------------------------------------------------------------------------

# インデックス作成用JSONの定義
create_index = {
    "settings": {
        "analysis": {
            # ---------------------------
            # Token Filter 定義
            # ---------------------------
            "filter": {
                "synonyms_filter": {  # 同義語フィルタの定義（検索側でのみ使用）
                    "type": "synonym",
                    "synonyms": [  # 等価展開の同義語リスト
                        "すし, スシ, 鮨, 寿司"
                        # 片方向に正規化したい場合の例:
                        # "スシ, 鮨, すし => 寿司"
                    ],
                    # 必要に応じて:
                    # "lenient": true   # フォーマット厳格性を緩める（検討用）
                    # "expand":  true   # 既定で true（相互展開）。false で片方向のみ
                }
                # 固有名詞を過正規化から保護したい場合の例（stemmer などの前段に置く）:
                # ,"ja_protected": {
                #     "type": "keyword_marker",
                #     "protected_words": ["メーラー","プレイヤー"]
                # }
            },
            # ---------------------------
            # Tokenizer 定義
            # ---------------------------
            "tokenizer": {
                "kuromoji_w_dic": {  # カスタム形態素解析の定義
                    "type": "kuromoji_tokenizer",  # ← 綴りは "kuromoji"
                    # ユーザー辞書（*.dic＝ビルド済み）を ES ノード側の参照可能パスで指定
                    "user_dictionary": "my_jisho.dic",
                }
            },
            # ---------------------------
            # Analyzer 定義
            # ---------------------------
            "analyzer": {
                # 検索用アナライザ（想起重視: 同義語を含める）
                "jpn-search": {
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer",  # 文字正規化（NFKC 等：全半角・互換文字）
                        "kuromoji_iteration_mark",  # 々/ゝ/ヽ 等の展開
                    ],
                    "tokenizer": "kuromoji_w_dic",  # ユーザー辞書付き kuromoji
                    "filter": [
                        "synonyms_filter",  # ← 同義語展開（検索側のみ）
                        "kuromoji_baseform",  # 活用語の原形化
                        "kuromoji_part_of_speech",  # 不要品詞除去（助詞/助動詞/記号 等）
                        "ja_stop",  # 日本語ストップワード除去
                        "kuromoji_number",  # 数表現の正規化（漢数字→算用数字 等）
                        "kuromoji_stemmer",  # カタカナ長音の正規化（コンピューター→コンピュータ）
                    ],
                },
                # 索引用アナライザ（精度重視: 通常は同義語を含めない）
                "jpn-index": {
                    "type": "custom",
                    "char_filter": ["icu_normalizer", "kuromoji_iteration_mark"],
                    "tokenizer": "kuromoji_w_dic",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "ja_stop",
                        "kuromoji_number",
                        "kuromoji_stemmer",
                        # ※ インデックス側は素直に保持。検索側で拡張（synonyms）する方針。
                    ],
                },
            },
        }
    }
}

# 日本語用インデックス名の定義
jp_index = "jp_index"

# 既存インデックスがあれば削除（検証用。運用ではエイリアス切替＋安全な移行手順が推奨）
if es.indices.exists(index=jp_index):
    es.indices.delete(index=jp_index)

# インデックス jp_index の生成（ここでは settings のみ）
# 実運用では mappings で対象フィールドに
#   "analyzer": "jpn-index", "search_analyzer": "jpn-search"
# を紐付けること（例: content フィールド）。
es.indices.create(index=jp_index, body=create_index)

# 【動作確認のヒント（コメント）】
# - _analyze で同義語展開を確認:
#   es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"寿司"})
#   → tokens に "寿司" と等価語（"すし","スシ","鮨" 等）が含まれることを確認（構成に依存）
# - 同義語更新フロー例:
#   新インデックス作成（新 synonyms）→ _reindex → エイリアス切替 → 旧を削除

In [ ]:
# 分析結果表示用関数（Elasticsearch の _analyze を用いて日本語トークン列を取得）
# -----------------------------------------------------------------------------
# 目的:
#   ・インデックス jp_index に定義済みの検索用アナライザ "jpn-search" を使って、
#     入力テキストを形態素解析＋各種フィルタ処理（原形化・品詞/ストップ語除去・数値/長音正規化・同義語展開 等）
#     に通し、得られた「語彙（token 文字列）」の配列を返す。
#
# 前提:
#   ・Elasticsearch 側で jp_index が作成済みで、settings.analysis.analyzer に "jpn-search" が存在すること。
#   ・Python 側では es（Elasticsearch クライアント）と jp_index 変数が有効であること。
#
# 理論メモ（何が起きるか）:
#   ・_analyze は「char_filter → tokenizer → filter」の順に処理を適用する。
#   ・"jpn-search" の想定チェーン（例）:
#       1) char_filter: icu_normalizer（全角/半角・互換文字の正規化）, kuromoji_iteration_mark（々/ゝ/ヽ の展開）
#       2) tokenizer : kuromoji（ユーザー辞書付き）
#       3) filter    : synonyms（同義語展開）→ baseform（原形化）→ POS/ja_stop（機能語除去）
#                      → number（数値正規化）→ stemmer（カタカナ長音の正規化）
#     これにより検索クエリ側の語彙が安定化し、インデックス側（通常は "jpn-index"）で格納された語彙と
#     BM25 等で比較される前提を検証できる。
# -----------------------------------------------------------------------------


def analyse_jp_text(text):
    # _analyze API に渡すリクエストボディを構築
    # - analyzer: 使用するアナライザ（検索時用 "jpn-search"）
    # - text    : 解析対象の文字列
    body = {"analyzer": "jpn-search", "text": text}

    # 指定インデックスのアナライザで解析を実行
    # 例外例:
    #   - RequestError(400): 指定アナライザが存在しない / 辞書読込失敗 / 設定不整合
    #   - ConnectionError  : ES ノード未起動・接続情報不備
    # 本関数では例外を上位へ伝播（学習用のため最小実装）。
    ret = es.indices.analyze(index=jp_index, body=body)

    # レスポンスは {"tokens": [ { "token": "...", "start_offset": ..., "end_offset": ..., "position": ..., "type": "..." }, ... ]}
    # 学習用として語彙文字列 "token" だけを抽出して返す
    tokens = ret["tokens"]
    tokens2 = [token["token"] for token in tokens]
    return tokens2


# 関数のテスト
# 期待例（設定依存）:
#   ・助詞や句読点は POS/ja_stop で落ちやすく、内容語が残る
#   ・「スシ/寿司/すし」のような表記揺れは icu_normalizer＋synonyms で吸収可能（設定していれば）
print(analyse_jp_text("関数のテスト"))

# （デバッグ補助：必要に応じて有効化）
# - 実際のトークン列（位置やオフセット）を見て解析チェーンの挙動を確認できる
# toks = es.indices.analyze(index=jp_index, body={"analyzer": "jpn-search", "text": "関数のテスト"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
# リスト 3.3.16 同義語のテスト（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" に定義した同義語（すし/スシ/鮨/寿司）が
#     クエリ側で展開され、表記ゆれを吸収してマッチ範囲を広げることを確認する。
#
# 前提:
#   ・インデックス jp_index が存在し、settings.analysis.analyzer に "jpn-search" が定義済み。
#   ・"jpn-search" の filter 先頭に `synonyms_filter`（"すし, スシ, 鮨, 寿司"）がある前提。
#   ・analyse_jp_text(text) は _analyze を呼び、"jpn-search" で解析した token 文字列配列を返す関数。
#
# 解析チェーン（想定）:
#   char_filter（icu_normalizer, kuromoji_iteration_mark）
#     → tokenizer（kuromoji + user_dictionary）
#       → filter（synonyms → baseform → POS/ja_stop → number → stemmer）
#   - 同義語は tokenizer の後に **同位置の代替語として展開**（等価展開; expand=true 相当）。
#   - その後、原形化・不要品詞/ストップ語除去・数値/長音正規化が順次適用される。
#
# 期待される挙動（出力は環境依存の一例。辞書・stoptags により差異あり）:
#   1) '寿司を食べたい'
#      - 「寿司」 → 同義語展開で {寿司, すし, スシ, 鮨} が同じ position に並ぶ
#      - 「を」   → 助詞のため POS/ja_stop で除去されやすい
#      - 「食べたい」→ 原形化で「食べる」
#      → 例: ['寿司','すし','スシ','鮨','食べる']
#
#   2) '私はスシが好きだ'
#      - 「スシ」 → 同義語展開で {スシ, すし, 寿司, 鮨}
#      - 「が」「だ」→ 助詞/助動詞のため除去されやすい
#      - 「好きだ」 → 原形化で「好き」
#      - 「私」     → 名詞として残る（stop 語に含めない限り）
#      → 例: ['私','スシ','すし','寿司','鮨','好き']
#
# メモ:
#   ・同義語は「検索側（jpn-search）」にのみ適用し、インデックス側（jpn-index）は素直に保持するのが定石。
#     これにより、変更時は新インデックス作成＋reindex＋エイリアス切替で安全に反映できる。
#   ・実運用で不要語（例: 一人称「私」）を落としたい場合は、`ja_stop` の拡張や `kuromoji_part_of_speech`
#     の `stoptags` 明示で制御するとよい。
# -----------------------------------------------------------------------------

# テスト1: 「寿司」を含む文。助詞は除去されやすく、「食べたい」は原形「食べる」に。
print(analyse_jp_text("寿司を食べたい"))

# テスト2: 半角/全角/漢字/かなを横断できるか確認（「スシ」が同義語展開で統合される想定）。
print(analyse_jp_text("私はスシが好きだ"))

# （デバッグ補助：必要ならコメント解除）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"私はスシが好きだ"})["tokens"]
# for t in toks:
#     # token と position/offset を見ると、同位置展開（synonyms）の挙動が把握しやすい
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
# リスト 3.3.12（改） マッピング定義（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・インデックス jp_index に対し、全文検索対象フィールド "content" のマッピングを設定する。
#   ・投入時は "jpn-index"（索引用アナライザ）、検索時は "jpn-search"（検索用アナライザ）を適用する。
#
# 背景/理論:
#   ・Elasticsearch v6 以降、全文検索用は "text"、完全一致/集計用は "keyword" が基本。
#   ・"analyzer" は「ドキュメント投入（indexing）」時に使われ、"search_analyzer" は
#     「クエリ解析」時に使われる（両者で処理を分離可能）。
#     - 本設計では、インデックス側は“素直に保持”（同義語なし）、
#       検索側で“想起を拡張”（同義語あり）することで観測性・運用性を確保する。
#
# 注意（重要）:
#   ・既に存在するフィールドに対して analyzer を変更することは不可（仕様）。必要なら
#     新インデックスを作成 → _reindex → エイリアス切替、の手順で移行する。
#   ・本マッピングは "content" のみを定義。他のフィールド（title, name 等）にも
#     解析要件がある場合は、それぞれに適切な analyzer / search_analyzer を明示する。
#   ・完全一致や並べ替え/集計が必要なら multi-fields を追加し、"content.raw": keyword
#     のように併用するのが定石。
# -----------------------------------------------------------------------------

mapping = {
    "properties": {
        "content": {
            # 全文検索対象のため "text" 型を指定（v5 の "string" は廃止済み）
            "type": "text",
            # ドキュメント投入時に使うアナライザ
            # - "jpn-index": kuromoji + 各種フィルタ（通常は同義語なし）
            # - 索引側をプレーンに保つことで、語彙の観測性・再構成容易性が高まる
            "analyzer": "jpn-index",
            # クエリ解析時に使うアナライザ
            # - "jpn-search": 同義語（すし/スシ/鮨/寿司）等で想起を拡張し、表記ゆれを吸収
            "search_analyzer": "jpn-search",
            # （必要に応じて追加例）
            # "fields": {
            #   "raw": {                 # 完全一致・ソート・集計用のサブフィールド
            #     "type": "keyword",
            #     "ignore_above": 256
            #     # normalizer を定義して大文字小文字統一なども可
            #   }
            # }
        }
    }
}

# マッピングの適用
# - 既存フィールドの analyzer 変更は不可。適用エラーや無視の原因になるため、移行は新インデックスで。
es.indices.put_mapping(index=jp_index, body=mapping)

# （確認のヒント：必要時にコメント解除）
# cur = es.indices.get_mapping(index=jp_index)
# from pprint import pprint
# pprint(cur[jp_index]["mappings"]["properties"].get("content"))

In [ ]:
# リスト 3.3.13（改）日本語文書の投入（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・前段で作成した日本語向けインデックス jp_index に、学習用のサンプル文書を投入する。
#   ・"content" フィールドにはマッピングで analyzer="jpn-index" / search_analyzer="jpn-search" を
#     付与済み（リスト 3.3.12）。投入時は jpn-index に従ってトークン化・正規化される。
#
# 理論メモ（索引時に何が起きるか）:
#   ・インデックス時（analyzer="jpn-index"）:
#       char_filter（icu_normalizer, iteration_mark）
#         → tokenizer（kuromoji + user_dictionary）
#           → filter（baseform, POS/ja_stop, number, stemmer）で**格納語彙**を生成。
#     これにより「全角/半角」「々/ゝ」「活用」「長音」などの表記ゆれが索引側で安定化する。
#   ・検索時（search_analyzer="jpn-search"）は上記に加えて synonyms を適用（表記ゆれの想起拡張）。
#     ⇒ 「スシ/すし/寿司/鮨」の横断が可能（同義語を jpn-search のみに置くのが定石）。
#
# 実務上の注意:
#   ・このスニペットでは "title" や "name" の明示マッピングは未設定 → 動的マッピングに依存する。
#     厳密な検索/集計要件があるなら、それぞれに text/keyword を設計しておくこと。
#   ・`es.index(id=...)` は同一 ID への再投入で**上書き**になる。衝突を避けたい場合は `op_type="create"`。
#   ・直後に検索する検証では `refresh="wait_for"` を付けるか、ループ後に `es.indices.refresh()` を呼ぶ。
#   ・大量投入は `elasticsearch.helpers.bulk` の利用で高速化・最適化できる。
# -----------------------------------------------------------------------------------

bodys = [
    {  # ドキュメント 0
        "title": "山田太郎の紹介",
        "name": {"last": "山田", "first": "太郎"},
        # 例: 「スシ」は icu_normalizer（互換/全半角）や stemmer と kuromoji により安定化して格納される
        "content": "スシが好物です。犬も好きです。",
    },
    {  # ドキュメント 1
        "title": "田中次郎の紹介",
        "name": {"last": "田中", "first": "次郎"},
        # 例: 「だいすき/大好き」の揺れは正規化＋原形化/stop である程度吸収
        #     固有名詞（はやぶさ）などは NEologd/ユーザー辞書の整備で分割の安定性が向上
        "content": "そばがだいすきです。新幹線はやぶさも好きです。",
    },
    {  # ドキュメント 2
        "title": "渡辺三郎の紹介",
        "name": {"last": "渡辺", "first": "三郎"},
        "content": "天ぷらが好きです。はやぶさのファンです。",
    },
]

# 文書投入:
# - enumerate の開始は 0。ここでは _id = 0, 1, 2 で投入される。
# - 既存 ID があると**上書き**される点に注意（検証用としては妥当）。
# - ES 8.x のクライアントでは `document=` 引数が推奨（`body=` は後方互換）。
for i, body in enumerate(bodys):
    es.index(index=jp_index, id=i, body=body)
    # 既存 ID があれば失敗させたい場合（衝突検出）:
    # es.index(index=jp_index, id=i, document=body, op_type="create")
    # 直後に検索する検証では各呼び出しに refresh を付ける:
    # es.index(index=jp_index, id=i, document=body, refresh="wait_for")

# バルク投入の雛形（大量データ時の推奨; ここでは実行しないため参考のみ）
# from elasticsearch import helpers
# actions = ({"_index": jp_index, "_id": i, "_source": doc} for i, doc in enumerate(bodys))
# helpers.bulk(es, actions, refresh="wait_for")

# 直後に検索する場合の明示リフレッシュ（wait_for を使わない場合の代替）
# es.indices.refresh(index=jp_index)

# 参考: 同義語テスト（検索側のみ synonyms を適用）
# - jpn-search に "すし, スシ, 鮨, 寿司" を定義済みなら、いずれの表記でもヒットが期待できる
# res = es.search(index=jp_index, query={"match": {"content": "寿司"}})
# print(res["hits"]["hits"][0]["_source"])

In [ ]:
# リスト 3.3.17 同義語による検索（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・"content" フィールドに対し、検索用アナライザ "jpn-search" を用いた全文検索（match）を行う。
#   ・"jpn-search" には同義語フィルタ（例: 「すし, スシ, 鮨, 寿司」）が含まれている前提。
#     → クエリ語「寿司」を投入すると、同位置に等価語が展開され、表記ゆれを横断してヒットが期待できる。
#
# 重要な理論ポイント（何が起きるか）:
#   1) match クエリは **全文検索**であり、クエリ文字列は search_analyzer（= "jpn-search"）で前処理される。
#      処理順: char_filter（icu_normalizer/iteration_mark）→ tokenizer（kuromoji）→
#               filter（synonyms → baseform → POS/ja_stop → number → stemmer）
#   2) `synonyms_filter` は等価展開（expand=true 相当）:
#      - 「寿司」→ 同位置に {寿司, すし, スシ, 鮨} を追加（同一 position）。
#      - これにより、インデックス側（通常 "jpn-index" で正規化済み）に格納されたどの表記ともマッチしやすくなる。
#   3) スコアリングは BM25 が既定:
#      - 展開後のトークンが文書側の語彙と一致するほどスコアが上がる。
#      - 語順一致が重要なら match_phrase、複数フィールドをまたぐなら multi_match を検討。
#
# 実務上の注意:
#   ・直前に index した文書がヒットしない場合は refresh 未反映の可能性があるため、
#     インデクシング側で refresh="wait_for" を使うか、ここで es.indices.refresh(index=jp_index) を行う。
#   ・返却件数は既定 size=10、厳密件数が必要な場合は track_total_hits=True を付ける（8.x では query=... 形式推奨）。
#   ・観測性のため、_analyze を併用してクエリ側トークンを確認すると原因追跡が速い（下に参考コード）。
# -----------------------------------------------------------------------------

# 検索条件の設定
# - クエリ語「寿司」は jpn-search の synonyms_filter により {寿司, すし, スシ, 鮨} へ等価展開される。
query = {"query": {"match": {"content": "寿司"}}}

# （参考・代替: 8.x 推奨の引数スタイル）
# res = es.search(index=jp_index, query={"match": {"content": "寿司"}}, size=20, track_total_hits=True)

# 検索実行
# - body=... は後方互換のため残している。8.x では query=... を推奨。
res = es.search(index=jp_index, body=query)

# 結果表示
# - ensure_ascii=False: 日本語を \u エスケープせず可読出力。
# - 実務では hits.hits から _id/_score/_source を抜粋してログ整形するのが見やすい。
import json

print(json.dumps(res, indent=2, ensure_ascii=False))

# -----------------------------------------------------------------------------
# （デバッグ支援：必要なときだけ使う参考コード。実行時はコメントを外す）
# 1) クエリ側の実トークンを確認（同義語展開・原形化・stop 適用の結果を見る）
# toks = es.indices.analyze(index=jp_index, body={"analyzer": "jpn-search", "text": "寿司"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))
#
# 2) 直前インデクシングの可視化（refresh）
# es.indices.refresh(index=jp_index)
#
# 3) 返戻の要約表示（閲覧性向上）
# hits = [{"_id": h["_id"], "_score": h["_score"], "_source": h["_source"]} for h in res["hits"]["hits"]]
# print(json.dumps({"total": res["hits"]["total"], "hits": hits}, indent=2, ensure_ascii=False))
# -----------------------------------------------------------------------------

In [ ]:
# リスト 3.3.18 「新幹線はやぶさ」と「はやぶさ」の分析結果（詳細コメント付き）
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" を通したときのトークン化結果を比較し、
#     固有表現（はやぶさ）の扱いと、複合語/連接の分割挙動を確認する。
#
# 理論メモ:
#   ・処理順は char_filter（icu_normalizer, iteration_mark）→ tokenizer（kuromoji+userdict）
#       → filter（synonyms → baseform → POS/ja_stop → number → stemmer）。
#   ・本例の語は記号・長音・反復記号を含まないため、主に **tokenizer（kuromoji）** と
#     **ユーザー辞書（my_jisho.dic）** の有無が結果を左右する。
#     - 「新幹線はやぶさ」: 通常は「新幹線」「はやぶさ」の2トークンに分割されやすい。
#       （ユーザー辞書で「新幹線はやぶさ」を一語登録していれば、1トークン化も可能）
#     - 「はやぶさ」: 固有名詞（列車名/探査機名/一般名詞「隼」）として1トークン化される想定。
#   ・同義語や原形化は本例にほぼ影響しない（名詞で活用無し、同義語も未定義なら no-op）。
#   ・固有表現の検索精度を上げたい場合:
#       1) ユーザー辞書で連接（「新幹線はやぶさ」）を1語として登録 → 索引側の分割安定化
#       2) フィールドを multi-fields 化（stemmer/stop 無しの厳密フィールドも併設）
#       3) クエリ側で match_phrase を併用（語順と近接を重視）
# ----------------------------------------------------------------------------------

# 比較1: 連接形（通常は「新幹線」「はやぶさ」の2トークン想定）
print(analyse_jp_text("新幹線はやぶさ"))

# 比較2: 単独形（通常は1トークン「はやぶさ」想定）
print(analyse_jp_text("はやぶさ"))

# （デバッグ補助：必要に応じてコメント解除）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"新幹線はやぶさ"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
# リスト 3.3.19 キーワード「はやぶさ」で検索（理論コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・全文検索（match）で "content" フィールドから「はやぶさ」を検索する。
#   ・検索側には search_analyzer="jpn-search" が適用される前提（同義語・原形化・品詞/stop 除去・
#     数値/長音正規化等がクエリ文字列に対して実行される）。
#
# 理論メモ（何が起こるか）:
#   ・match は **token-level** の全文検索。クエリ文字列は jpn-search の処理
#     [char_filter → kuromoji → filter] を通ってトークン化され、インデックス側（通常は jpn-index）
#     で格納済みの語彙と BM25（TF・IDF・文書長補正）でスコアリングされる。
#   ・「はやぶさ」は一般名詞/固有名詞（列車名・探査機名等）として 1 トークン化されることが多いが、
#     kuromoji の辞書/ユーザー辞書に依存する。連接語（例: 「新幹線はやぶさ」）を 1 語として扱いたい
#     場合はユーザー辞書に登録するか、クエリ側で match_phrase を用いて語順・近接を強調する。
#   ・同義語は jpn-search に定義していればクエリ側で展開される（等価展開）。未定義なら no-op。
#
# 実務的な注意:
#   ・直前に index した文書がヒットしない場合は refresh の問題。検証時は index 時に
#     refresh="wait_for" を付けるか、検索前に es.indices.refresh(index=jp_index) を実行。
#   ・返戻件数は既定 size=10、厳密件数が必要なら track_total_hits=True を指定（下に代替例のコメント）。
#   ・完全一致（非解析）を行いたい用途は keyword フィールド + term クエリを使用する。
#   ・語順/フレーズ一致を重視する場合は match_phrase（必要に応じて slop を調整）。
# -----------------------------------------------------------------------------

# 検索条件の設定
query = {
    "query": {"match": {"content": "はやぶさ"}}  # ← クエリ側で jpn-search が適用される
}

# 検索実行
# ※ 8.x クライアントでは body=... より query=... の使用が推奨（後方互換で body も可）。
#    厳密件数や返戻件数をコントロールしたい場合は下の代替例を参照。
res = es.search(index=jp_index, body=query)

# 結果表示
# - ensure_ascii=False: 日本語を \u エスケープせずに出力
import json

print(json.dumps(res, indent=2, ensure_ascii=False))

# -----------------------------------------------------------------------------
# （代替/拡張例: 必要に応じてコメントアウトを外して使用）
# 1) 厳密件数と返戻サイズを明示（推奨）
# res = es.search(
#     index=jp_index,
#     query={"match": {"content": "はやぶさ"}},
#     size=20,
#     track_total_hits=True,
#     _source=["title","name.last","content"]
# )
# hits = [
#     {"_id": h["_id"], "_score": h["_score"], "_source": h["_source"]}
#     for h in res["hits"]["hits"]
# ]
# print(json.dumps({"total": res["hits"]["total"], "hits": hits}, indent=2, ensure_ascii=False))
#
# 2) フレーズ一致（語順・近接を重視）
# res = es.search(
#     index=jp_index,
#     query={"match_phrase": {"content": "新幹線 はやぶさ"}},
#     size=20,
#     track_total_hits=True
# )
#
# 3) クエリ側トークンの可視化（_analyze）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"はやぶさ"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))
#
# 4) 直前投入の可視化
# es.indices.refresh(index=jp_index)
# -----------------------------------------------------------------------------